In [2]:
# start coding here
i_ids = snakemake.input.bondugula_ids
o_csv = snakemake.output.dataset
o_missing = snakemake.output.missing

In [18]:
import os

import pandas as pd


def _get_csv_path(_id):
    fpath = f"data/isSwitch.dataset/{_id[0:2]}/{_id}.csv"
    if os.path.isfile(fpath):
        return fpath
    else:
        return False

with open(i_ids, "r") as f:
    _ids = list(set([f"{line[1:5]}_{line[5].upper()}" for line in f.readlines()]))

# initate first dataframe
full_df = None
missing_count = 0

for i, _id in enumerate(_ids[1:]):
    print(f"Running {i+1} of {len(_ids)}. Missing {missing_count} {' '*8}", end="\r")
    csv_path = _get_csv_path(_id)
    if csv_path:
        if isinstance(full_df, pd.DataFrame):
            new_df = pd.read_csv(csv_path, index_col=0)
            new_df["protein"] = _id
            full_df = pd.concat([full_df, new_df])
            
        else:
            full_df = pd.read_csv(csv_path, index_col=0)
            full_df["protein"] = _id
    else:
        missing_count += 1
        with open(o_missing, "w+") as f:
            f.write(f"{_id}\n")



In [21]:
# save to file
full_df.to_csv(o_csv)